In [7]:
import xgboost
print(xgboost.__version__)

0.71


In [3]:
import sys
print(sys.version)

3.5.5 |Anaconda, Inc.| (default, Apr  7 2018, 04:52:34) [MSC v.1900 64 bit (AMD64)]


In [5]:
import numpy as np
print(np.version.version)

1.13.3


In [6]:
import scipy
print(scipy.__version__)

1.0.1


In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

In [9]:
from sklearn.metrics import accuracy_score, log_loss

In [10]:
from sklearn.model_selection import cross_val_predict, cross_val_score, train_test_split

In [11]:
from sklearn.datasets import make_classification

In [23]:
# creates toy data set of classification
X, y = make_classification(n_samples=1000,n_features=20, n_informative=8, n_redundant=3, n_repeated=2)

In [24]:
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [25]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 402, 1: 398})
Counter({0: 103, 1: 97})


<h2> Decision Tree Classfier </h2>

In [28]:
dcsn_clf  = DecisionTreeClassifier()
dcsn_clf.fit(X_train, y_train)
y_predict = dcsn_clf.predict(X_test)
print("Accuracy is %.2f"%accuracy_score(y_predict, y_test))
print("Log loss score is %.2f"%log_loss(y_predict, y_test))

Accuracy is 0.81
Log loss score is 6.74


Log loss is more becase classfier out 1 or 0 not probabilities

In [34]:
print(dcsn_clf.predict(X_test[0:5]))
print("Actual Below")
print(y_test[0:5])

[0 0 0 1 1]
Actual Below
[0 0 0 1 0]


<h3>ADA boost Classifier</h3>

In [39]:
ada_boost  = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=500, algorithm='SAMME')
ada_boost.fit(X_train, y_train)
y_predict = ada_boost.predict(X_test)
y_predict_prob = ada_boost.predict_proba(X_test)
print("Accuracy is %.2f"%accuracy_score(y_predict, y_test))
print("Log loss score is %.2f"%log_loss(y_test, y_predict_prob))

Accuracy is 0.88
Log loss score is 0.68


In [40]:
print(ada_boost.predict_proba(X_test[0:5]))
print("Actual Below")
print(y_test[0:5])

[[ 0.51063695  0.48936305]
 [ 0.51180713  0.48819287]
 [ 0.51335707  0.48664293]
 [ 0.49996298  0.50003702]
 [ 0.50164438  0.49835562]]
Actual Below
[0 0 0 1 0]


<h3> Gradient Boosting Classfier </h3>

In [42]:
gb_clf = GradientBoostingClassifier(max_depth=1, n_estimators=500, warm_start=True)
gb_clf.fit(X_train, y_train)
y_predict = gb_clf.predict(X_test)
y_predict_prob = gb_clf.predict_proba(X_test)
print("Accuracy is %.2f"%accuracy_score(y_predict, y_test))
print("Log loss score is %.2f"%log_loss(y_test, y_predict_prob))

Accuracy is 0.89
Log loss score is 0.29


In [44]:
print(gb_clf.predict_proba(X_test[0:5]))
print("Actual Below")
print(y_test[0:5])

[[ 0.80537146  0.19462854]
 [ 0.96541166  0.03458834]
 [ 0.95434788  0.04565212]
 [ 0.09690193  0.90309807]
 [ 0.40532594  0.59467406]]
Actual Below
[0 0 0 1 0]


If you observe one intersting thing here is probabilities are very well seperated, but in Ada boost they are very close, that is the reason why adaboost has more log loss than gradient boosting 

<h3>Xgboost Standard Interface</h3>

In [49]:
dtrain = xgboost.DMatrix('../xg_boost_data/agaricus.txt.train')

In [50]:
dtest = xgboost.DMatrix('../xg_boost_data/agaricus.txt.test')

In [54]:
dtrain.num_row()

6513

In [56]:
dtest.num_row()

1611

In [59]:
dtrain.get_label()

array([ 1.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [60]:
params = {
    'objective':'binary:logistic',
    'max_depth':2,
    'silent':1,
    'eta':1
}
num_rounds = 5
xgboost.train(params, dtrain, num_boost_round=num_rounds)

In [63]:
# In this interesting thing you can send the watch list as well and you can check the performance
watchlist = [(dtrain,'train'), (dtest, 'test')]
bst = xgboost.train(params, dtrain, num_rounds, watchlist)

[0]	train-error:0.046522	test-error:0.042831
[1]	train-error:0.022263	test-error:0.021726
[2]	train-error:0.007063	test-error:0.006207
[3]	train-error:0.0152	test-error:0.018001
[4]	train-error:0.007063	test-error:0.006207


In [64]:
preds_prob = bst.predict(dtest)
labels = dtest.get_label()
preds = preds_prob > 0.5 # threshold
correct = 0

for i in range(len(preds)):
    if (labels[i] == preds[i]):
        correct += 1

print('Predicted correctly: {0}/{1}'.format(correct, len(preds)))
print('Error: {0:.4f}'.format(1-correct/len(preds))) 

Predicted correctly: 1601/1611
Error: 0.0062


## scikit Learn Interface

In [65]:
from xgboost import XGBClassifier

In [66]:
from sklearn.datasets import load_svmlight_files

In [67]:
X_train, y_train, X_test, y_test = load_svmlight_files(('../xg_boost_data/agaricus.txt.train','../xg_boost_data/agaricus.txt.test'))

In [68]:
X_train.shape

(6513, 126)

In [69]:
X_test.shape

(1611, 126)

In [76]:
xgb_clf = XGBClassifier(n_estimators=5, learning_rate=1.0, objective='binary:logistic',silent=1, max_depth=2)

In [77]:
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=1.0, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=5,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=1)

In [78]:
y_predict = xgb_clf.predict(X_test)
accuracy_score(y_predict, y_test)

0.99379267535692117